In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv


In [2]:
import sklearn
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

data = pd.read_csv("../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv")
data.head()

,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,4436033771,0,75,4,0,6,6,6,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4


In [3]:
X = data.copy()
X = X.drop(columns=['gameId', 'blueWins'])
y = data['blueWins']

# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [4]:
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred[:10]

array([1, 0, 1, 1, 1, 0, 1, 0, 0, 0])

In [5]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[670, 277],
       [315, 714]])

In [ ]:
# xgb_model = XGBClassifier(silent=False, 
#                               booster='gbtree',
#                               scale_pos_weight=1,
#                               learning_rate=0.01,  
#                               colsample_bytree = 0.4,
#                               subsample = 0.8,
#                               objective='binary:logistic', 
#                               n_estimators=500, 
#                               max_depth=6, 
#                               gamma=10, 
#                               seed=777)

# hr_pred = xgb_model.fit(X_train, y_train).predict(X_test)
# print(classification_report(y_test, hr_pred))
# print(confusion_matrix(y_test, hr_pred))

In [7]:
from sklearn.model_selection import GridSearchCV

# XGBoost 분류기 생성
xgb_clf = XGBClassifier(learning_rate=0.02, n_estimator=1500, colsample_bytree=0.8)

# 초모수 격자생성
xgb_param_grid = {'max_depth': [3, 5, 7], 
                  'subsample': [0.6, 0.8, 1.0]}

# Create a GridSearchCV object
hr_grid = GridSearchCV(estimator=xgb_clf,
                       param_grid=xgb_param_grid,
                       scoring='f1',
                       cv=5,
                       refit=True, 
                       return_train_score=True)

hr_grid.fit(X_train, y_train)
hr_grid_df = pd.DataFrame(hr_grid.cv_results_)

hr_grid_df.loc[:, ['mean_test_score', "params"]]

,mean_test_score,params
0,0.720654,"{'max_depth': 3, 'subsample': 0.6}"
1,0.722506,"{'max_depth': 3, 'subsample': 0.8}"
2,0.721687,"{'max_depth': 3, 'subsample': 1.0}"
3,0.720169,"{'max_depth': 5, 'subsample': 0.6}"
4,0.721874,"{'max_depth': 5, 'subsample': 0.8}"
5,0.720251,"{'max_depth': 5, 'subsample': 1.0}"
6,0.721058,"{'max_depth': 7, 'subsample': 0.6}"
7,0.721576,"{'max_depth': 7, 'subsample': 0.8}"
8,0.715104,"{'max_depth': 7, 'subsample': 1.0}"


In [12]:
best_score = hr_grid.best_score_
best_row = hr_grid.best_index_

best_max_depth     = hr_grid.best_params_["max_depth"]
best_max_subsample = hr_grid.best_params_["subsample"]

print(f"Best score        : {best_score}")
print(f"Best max depth    : {best_max_depth}")
print(f"Best max subsample: {best_max_subsample}")

Best score        : 0.7225060580825773
Best max depth    : 3
Best max subsample: 0.8
